# 雀魂網頁相關(?)
處理 python bot -> 網頁雀魂的各種雜事...

(因為這不太重要+我懶所以)
座標目前都用我本機量的絕對座標，理論上不同電腦會有偏差
如果要量的話可以用下面這段程式碼抓 x,y 座標(但挺麻煩qwq)

In [13]:
'''
from pynput import mouse, keyboard

def on_click(x, y, button, pressed):
    if pressed:  # 滑鼠點擊時打印座標
        print(f"滑鼠點擊座標：x={x}, y={y}")

def on_press(key):
    try:
        # 如果按下的鍵是 'e'，退出程式
        if key.char == 'e':
            print("偵測到按下 'e' 鍵，結束程式。")
            return False  # 停止鍵盤監聽
    except AttributeError:
        # 忽略非字母按鍵
        pass

# 啟動滑鼠監聽器
with mouse.Listener(on_click=on_click) as mouse_listener, keyboard.Listener(on_press=on_press) as keyboard_listener:
    print("請點擊滑鼠查看座標，按下 'e' 鍵結束程式。")
    keyboard_listener.join()  # 等待鍵盤監聽器退出
    mouse_listener.stop()  # 停止滑鼠監聽器

'''

'\nfrom pynput import mouse, keyboard\n\ndef on_click(x, y, button, pressed):\n    if pressed:  # 滑鼠點擊時打印座標\n        print(f"滑鼠點擊座標：x={x}, y={y}")\n\ndef on_press(key):\n    try:\n        # 如果按下的鍵是 \'e\'，退出程式\n        if key.char == \'e\':\n            print("偵測到按下 \'e\' 鍵，結束程式。")\n            return False  # 停止鍵盤監聽\n    except AttributeError:\n        # 忽略非字母按鍵\n        pass\n\n# 啟動滑鼠監聽器\nwith mouse.Listener(on_click=on_click) as mouse_listener, keyboard.Listener(on_press=on_press) as keyboard_listener:\n    print("請點擊滑鼠查看座標，按下 \'e\' 鍵結束程式。")\n    keyboard_listener.join()  # 等待鍵盤監聽器退出\n    mouse_listener.stop()  # 停止滑鼠監聽器\n\n'

### Import & Parameters

In [14]:
from selenium import webdriver
from pynput import mouse, keyboard
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyautogui
import time
import json
import sys
from enum import Enum, auto
class GameState(Enum):
    Loading = auto()
    Home = auto()
    InGame = auto()
OP_INTERVAL = 1 # click interval
current_state = None
current_hand = None
driver = None
auto_running = False
main_running = True

### Start Game & Auto Login

開啟一個 Chrome 瀏覽器連到雀魂並登入測試帳號

In [15]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
def start_game():
    global current_state, driver
    current_state = GameState.Loading
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get("https://game.maj-soul.com")
    driver.maximize_window()
    time.sleep(10)

    pyautogui.moveTo(1200, 430)
    pyautogui.click()
    pyautogui.write("paopaodmm@gmail.com", interval=0.05)
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(1200, 520)
    pyautogui.click()
    pyautogui.write("paopao123", interval=0.05)
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(1200, 670)
    pyautogui.click()

    # wait for the page to load
    time.sleep(12)

    # close the pop-up
    pyautogui.moveTo(1640, 370)
    pyautogui.click()
    time.sleep(OP_INTERVAL)
    current_state = GameState.Home
    print("Game started")

### Close browser

In [16]:
def close_browser():
    global driver
    driver.quit()
    driver = None
    print("Browser closed.")

### 自動創機器人房間

必須在首頁才能呼叫

In [17]:
def enter_bot_game():
    global current_state
    if current_state != GameState.Home:
        print("Not in home state, Abort")
        return
    # create bot room
    pyautogui.moveTo(1280, 750)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(1280, 500)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(965, 920)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    # add 3 bots
    pyautogui.moveTo(700, 670)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(1000, 670)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(1300, 670)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    # start the game
    pyautogui.moveTo(1080, 950)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    # wait for the game to start
    # time.sleep(15)
    current_state = GameState.InGame
    print("Bot game started")

### Leave game

In [18]:
def leave_bot_game():
    global current_state
    if current_state != GameState.InGame:
        print("Not in game state, Abort")
        return
    
    pyautogui.moveTo(1645, 235)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(825, 710)
    pyautogui.click()
    time.sleep(OP_INTERVAL)
    
    # Return to the main page time
    # time.sleep(5)
    current_state = GameState.Home
    print("Bot game left")

### 嵌入 js code
github上的code ><

In [19]:
def inject_js():
    global driver
    # with open("AlphaJong_1.3.1.user.js", "r", encoding="utf-8") as js_file:
    #     js_code = js_file.read()
    with open("web.js", "r", encoding="utf-8") as js_file:
        js_code = js_file.read()
    
    driver.execute_script(js_code)
    print("JavaScript 已成功嵌入！")

In [20]:
def filter_hand_data(data):
    filtered_data = {'dora': data['dora'], 'type': data['type'], 'index': data['index']}
    return filtered_data
def update_hand_data(new_data):
    global current_hand
    filtered_data = [filter_hand_data(tile) for tile in new_data]
    current_hand = filtered_data

def visualize_tile(tile):
    dora, tile_type, index = tile['dora'], tile['type'], tile['index']
    if tile_type == 0:
        tile_type = '筒'
    elif tile_type == 1:
        tile_type = '萬'
    elif tile_type == 2:
        tile_type = '條'
    else:
        tile_type = '字'
    return f"{index} {tile_type} {'Dora' if dora else ''}"

def print_current_hand():
    global current_hand
    for i, tile in enumerate(current_hand):
        print(f"tile {i}: {visualize_tile(tile)}")

### Press Event


In [21]:
def on_press(key):
    global auto_running, main_running, driver
    hand_data = None
    try:
        # 基本上執行完之前都會被 block 住
        if key.char == 'e':
            print("'e' key pressed, leaving bot game.")
            leave_bot_game()
        elif key.char == 's':
            print("'s' key pressed, starting bot game.")
            enter_bot_game()
        elif key.char == 'q':
            print("'q' key pressed, closing browser.")
            close_browser()
            main_running = False
            sys.exit(0)
        elif key.char == 'h':
            print("'h' key pressed, Helloworld.")
            driver.execute_script("helloword();")
        elif key.char == 'g':
            print("'g' key pressed, get Hand Data.")
            hand_data = driver.execute_script("return GetHandData();")
            if hand_data is None:
                print("手牌資料未返回，請檢查 JavaScript 函數的返回值！")
            else:
                update_hand_data(hand_data)
        elif key.char == 'p':
            print("'p' key pressed, print hand data.")
            print_current_hand()
            
        elif key.char == 'd':
            print("'d' key pressed, discard tile.")
            # before discard
            # hand_data = driver.execute_script("return GetHandData();")
            # update_hand_data(hand_data)
            # print("Before discard:")
            # print_current_hand()
            result = driver.execute_async_script("""
                // Selenium 的回調函數
                const done = arguments[arguments.length - 1];

                // 執行 discard 並處理 Promise
                discard(window.ownHand)
                    .then(tile => done(tile))        // 返回丟出的牌
                    .catch(err => done(err.message)); // 捕獲錯誤並返回錯誤訊息
            """)
            print(f"丟出的牌: {visualize_tile(filter_hand_data(result))}")
        elif key.char == 't':
            print("'t' key pressed, Toggle Auto Play.")
            driver.execute_script("window.Auto_run = !window.Auto_run;")
            auto_running = not auto_running
    except AttributeError:
        pass

## Import Model

In [22]:
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow
import numpy as np
from chi_pon_kong_RL import ChiPongKongEnvironment
from chi_pon_kong_RL import RLAgent
from chi_pon_kong_RL import train_agent
rl_env = ChiPongKongEnvironment()
rl_agent = RLAgent(action_space=["chi", "pong", "kong", "skip"])
train_agent()
class Model_NN():
    def __init__(self) -> None:
        self.model = Sequential([Dense(64, activation='relu', input_shape=(34,)),
                                 Dense(128, activation='relu'),
                                 Dense(256, activation='relu'),
                                 Dropout(0.2),
                                 Dense(34, activation='softmax')
                                ])
        self.model.compile(optimizer= tensorflow.keras.optimizers.Adam(learning_rate=1e-3),loss='categorical_crossentropy', metrics=['accuracy'])       
        self.model.load_weights('checkpoints\model_epoch_10.h5')


    #確保輸出只有輸入

    def filter_prediction(self, input_tiles, prediction):
        """
        过滤预测结果，确保输出的牌在玩家手牌内。
        
        Args:
            input_tiles (np.array): 玩家手牌 (长度为34的向量)
            prediction (np.array): 模型的输出 (长度为34的概率向量)
        
        Returns:
            int: 经过过滤后的输出牌索引
        """
        legal_tiles = np.where(input_tiles > 0)[1]  # 找到手牌中合法牌的索引
        prediction = prediction.reshape(34,)
        filtered_prediction = np.zeros_like(prediction)
        
        #print(f"合法牌索引：{legal_tiles}")
        #print(f"原始预测值：{prediction}")
        
        filtered_prediction[legal_tiles] = prediction[legal_tiles]
        
        #print(f"过滤后的预测值：{filtered_prediction}")
        
        return np.argmax(filtered_prediction)

    #棄牌模型

    def discard(self, hands_input):
        # 初始化手牌
        hands = np.zeros(34)  # 假设有 34 种牌

        # 根据 hands_input 中的牌来填充 hands 数组
        for tile in hands_input:
            hands[tile['type'] * 9 + tile['index'] - 1] += 1

        # 让模型预测
        hands = hands.reshape(1, 34)  # 使其成为符合模型要求的形状
        out = self.filter_prediction(hands, self.model.predict(hands))


        # 找到并返回对应的弃牌
        for i, tile in enumerate(hands_input):
            if tile['type'] * 9 + tile['index'] - 1 == out:
                return i
model = Model_NN()

<>:21: SyntaxWarning: invalid escape sequence '\m'
<>:21: SyntaxWarning: invalid escape sequence '\m'
C:\Users\danny\AppData\Local\Temp\ipykernel_45124\2436646436.py:21: SyntaxWarning: invalid escape sequence '\m'
  self.model.load_weights('checkpoints\model_epoch_10.h5')


Episode 0: Total Reward = -40.10, Avg Reward = -40.10
Episode 100: Total Reward = 592.70, Avg Reward = 5.87
Episode 200: Total Reward = 1189.80, Avg Reward = 5.92
Episode 300: Total Reward = 1705.30, Avg Reward = 5.67
Episode 400: Total Reward = 2225.90, Avg Reward = 5.55
Episode 500: Total Reward = 2726.10, Avg Reward = 5.44
Episode 600: Total Reward = 3246.70, Avg Reward = 5.40
Episode 700: Total Reward = 3746.90, Avg Reward = 5.35
Episode 800: Total Reward = 4236.90, Avg Reward = 5.29
Episode 900: Total Reward = 4726.90, Avg Reward = 5.25


## Test

這邊可以加入更多測試項目

像是按下 's' 開啟一場 bot game，按下 'e' 跳回首頁等等

In [23]:

def main():
    global auto_running, driver, main_running
    start_game()
    inject_js()
    with keyboard.Listener(on_press=on_press) as keyboard_listener:
        print(main_running)
        while main_running:
            if not auto_running:
                print("Auto_run is False")
                time.sleep(0.5)
                continue
            # get operations list
            operations = driver.execute_script("return getOperationList(1);")
            operations = json.loads(operations)
            print(f"Main Loop {operations}")
            
            if not operations or len(operations) == 0:
                print("Waiting for own turn.")
                time.sleep(0.5)
                continue
            
            main_own_turn()
            time.sleep(1)

def main_own_turn():
    global driver, current_hand, rl_agent, rl_env  # 加入 RL 代理和环境
    if driver.execute_script("return window.threadIsRunning;"):
        return
    driver.execute_script("window.threadIsRunning = true;")
    driver.execute_script("GetHandData();")
    
    operations = driver.execute_script("return getOperationList(1);")
    operations = json.loads(operations)
    print("##### OWN TURN #####")
    
    for operation in operations:
        if driver.execute_script("return getOperationList(1).length;") == 0:
            break
        operation_type = operation.get('type')
        
        if operation_type == driver.execute_script("return getOperations().an_gang;"):
            driver.execute_script(f"callAnkan({0}, {operation['combination']});")
        elif operation_type == driver.execute_script("return getOperations().add_gang;"):
            driver.execute_script(f"callShouminkan({0});")
        elif operation_type == driver.execute_script("return getOperations().zimo;"):
            driver.execute_script("callTsumo();")
        elif operation_type == driver.execute_script("return getOperations().rong;"):
            driver.execute_script("callRon();")
        elif operation_type == driver.execute_script("return getOperations().jiuzhongjiupai;"):
            driver.execute_script(f"callAbortiveDraw({0});")
    
    for operation in operations:
        if driver.execute_script("return getOperationList(1).length;") == 0:
            break
        operation_type = operation.get('type')
        
        if operation_type == driver.execute_script("return getOperations().dapai;"):
            hand_data = driver.execute_script("return GetHandData();")
            update_hand_data(hand_data)
            richi = False
            comb = None
            for op in operations:
                if op['type'] == driver.execute_script("return getOperations().liqi;"):
                    richi = True
                    comb = op['combination']
                    break
            if richi:
                # Todo: ask richi model, for now just call richi
                print("Richi!"+str(comb))
                for c in comb:
                    print(f"Combination: {c}")
                    js_argument = json.dumps(c)
                    driver.execute_script(f"callRiichi({js_argument});")
                    break
            else:
                tile_idx = model.discard(current_hand)
                driver.execute_script(f"discard_idx({tile_idx});") 
                # driver.execute_script("discard(window.ownHand);")
        elif operation_type == driver.execute_script("return getOperations().eat;"):
            # RL 决策是否执行吃
            action = rl_agent.choose_action(rl_env.state)  # 获取 RL 动作
            if action == "chi":
                driver.execute_script(f"callEat({1});")
            else:
                driver.execute_script(f"callEat({0});")
        elif operation_type == driver.execute_script("return getOperations().peng;"):
            # RL 决策是否执行碰
            action = rl_agent.choose_action(rl_env.state)  # 获取 RL 动作
            if action == "pong":
                driver.execute_script(f"callPeng({1});")
            else:
                driver.execute_script(f"callPeng({0});")
        elif operation_type == driver.execute_script("return getOperations().ming_gang;"):
            # RL 决策是否执行明杠
            action = rl_agent.choose_action(rl_env.state)  # 获取 RL 动作
            if action == "kong":
                driver.execute_script(f"callDaiminkan({1});")
            else:
                driver.execute_script(f"callDaiminkan({0});")
    
    print("Own turn completed.")
    driver.execute_script("window.threadIsRunning = false;")


In [ ]:
main()